<a href="https://colab.research.google.com/github/leeje008/ML_final_project/blob/main/final_code/Tunning_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import  confusion_matrix,recall_score,precision_score,accuracy_score
from imblearn.metrics import geometric_mean_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier  
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE




def get_clf_eval(y_test,pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc = roc_auc_score(y_test, pred)
    g_mean = geometric_mean_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:4f}, 정밀도 {1:4f}, 재현율 {2:4f}, AUC {3:4f},G_MEAN {4:4f}'.format(accuracy , precision , recall,roc,g_mean))
    

def get_model_train_eval(model,ftr_train = None, ftr_test = None, tgt_train = None, tgt_test = None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)



import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Loan_Train.csv')
df.head()

df = df.drop('Id',axis = 1)


from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()

data = df


for e in data.columns:
    if data[e].dtype == 'object':
        labelEncoder.fit(list(data[e].values))
        data[e] = labelEncoder.transform(data[e].values)
        
        # Accommodate the data that has been changed
        df = data
        
y = df.Risk_Flag
X = df.drop('Risk_Flag', axis=1)


X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size=0.3,
                                                   random_state=101)



# Hyperpameter tunning RF

In [ ]:
! pip install smote-variants

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN
import smote_variants as sv
from sklearn.metrics import roc_auc_score, make_scorer

In [ ]:
start = time.time()


pipeline = Pipeline(steps= [("SMOTE", SMOTE(random_state = 101)),
                            ("RandomForestClassifier", RandomForestClassifier())
                            ])

param_grid = {
    "SMOTE__sampling_strategy": [0.5,0.75,1],
    "RandomForestClassifier__n_estimators": [100,200,300,400,500]
              }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline.fit(X_train, y_train)

end = time.time()

print(f"{end - start:.5f} sec")

# Store the best model
smote_best_model = gs_pipeline.best_estimator_

y_validation_preds = smote_best_model.predict(X_test)
roc_auc_score(y_test, y_validation_preds)

## 결과물 저장

In [ ]:
scores_df = pd.DataFrame(gs_pipeline.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/RF_SMOTE.csv',index = False)

# RF ADA

In [ ]:
start = time.time()


pipeline = Pipeline(steps= [("ada", ADASYN(random_state = 101)),
                            ("RandomForestClassifier", RandomForestClassifier())
                            ])

param_grid = {
    "ada__sampling_strategy": [0.5,0.75,1],
    "RandomForestClassifier__n_estimators": [100,200,300,400,500]
              }



gs_pipeline_1 = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline_1.fit(X_train, y_train)

end = time.time()

print(f"{end - start:.5f} sec")

# Store the best model


scores_df = pd.DataFrame(gs_pipeline_1.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/RF_ADA.csv',index = False)

# RF SVR

In [ ]:
start = time.time()


pipeline = Pipeline(steps= [("SVR", sv.distance_SMOTE()),
                            ("RF", RandomForestClassifier())
                            ])

param_grid = {
    "SVR__proportion": [0.5,0.75,1],
    "RF__n_estimators": [100,200,300,400,500]
              }



gs_pipeline_1 = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline_1.fit(np.array(X_train), np.array(y_train))

end = time.time()

print(f"{end - start:.5f} sec")

# Store the best model


scores_df = pd.DataFrame(gs_pipeline_1.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/RF_SVR.csv',index = False)

svr_best_model = gs_pipeline_1.best_estimator_


y_validation_preds = svr_best_model.predict(np.array(X_test))

roc_auc_score(y_test, y_validation_preds)

# LGB hyperparameter tunning
# LGB SMOTE

In [ ]:
start = time.time()


pipeline = Pipeline(steps= [(("SMOTE",  SMOTE(random_state = 101))),
                             ("LGBClassifier", LGBMClassifier(boost_from_average = False))
                             ])

param_grid = {
    "SMOTE__sampling_strategy": [0.5,0.75,1],
    "LGBClassifier__n_estimators": [100,200,300,400,500]
              }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline.fit(X_train, y_train)

end = time.time()

print(f"{end - start:.5f} sec")



scores_df = pd.DataFrame(gs_pipeline.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/LGB_SMOTE.csv',index = False)

# LGB ADASYN


In [ ]:
start = time.time()


pipeline = Pipeline(steps= [(("ada",  ADASYN(random_state = 101))),
                             ("LGBClassifier", LGBMClassifier(boost_from_average = False))
                             ])

param_grid = {
    "ada__sampling_strategy": [0.5,0.75,1],
    "LGBClassifier__n_estimators": [100,200,300,400,500]
              }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline.fit(X_train, y_train)

end = time.time()

print(f"{end - start:.5f} sec")



scores_df = pd.DataFrame(gs_pipeline.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/LGB_ADASYN.csv',index = False)

# LGB SVR

In [ ]:
start = time.time()


pipeline = Pipeline(steps= [("SVR", sv.distance_SMOTE()),
                            ("LGB", LGBMClassifier())
                            ])

param_grid = {
    "SVR__proportion": [0.5,0.75,1],
    "LGB__n_estimators": [100,200,300,400,500]
              }



gs_pipeline_1 = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline_1.fit(np.array(X_train), np.array(y_train))

end = time.time()

print(f"{end - start:.5f} sec")

# Store the best model


scores_df = pd.DataFrame(gs_pipeline_1.cv_results_)
scores_df.to_csv('/content/drive/MyDrive/LGB_SVR.csv',index = False)

svr_best_model = gs_pipeline_1.best_estimator_


y_validation_preds = svr_best_model.predict(np.array(X_test))

roc_auc_score(y_test, y_validation_preds)

# KNN SVR

In [ ]:
start = time.time()


pipeline = Pipeline(steps= [("SVR", sv.distance_SMOTE()),
                            ("KNeighborsClassifier", KNeighborsClassifier())
                            ])

param_grid = {
    "SVR__proportion": [0.5,0.75,1],
    "KNeighborsClassifier__n_neighbors": list(range(10,101,10))
              }



gs_pipeline_1 = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 5)
gs_pipeline_1.fit(np.array(X_train), np.array(y_train))

end = time.time()

print(f"{end - start:.5f} sec")

svr_best_model = gs_pipeline_1.best_estimator_


y_validation_preds = svr_best_model.predict(np.array(X_test))

roc_auc_score(y_test, y_validation_preds)

In [ ]:
def plot_grid_search(cv_results, grid_param_1, grid_param_2, name_param_1, name_param_2):
    # Get Test Scores Mean and std for each grid search
    scores_mean = cv_results['mean_test_score']
    scores_mean = np.array(scores_mean).reshape(len(grid_param_2),len(grid_param_1))

    scores_sd = cv_results['std_test_score']
    scores_sd = np.array(scores_sd).reshape(len(grid_param_2),len(grid_param_1))

    # Plot Grid search scores
    _, ax = plt.subplots(1,1)

    # Param1 is the X-axis, Param 2 is represented as a different curve (color line)
    for idx, val in enumerate(grid_param_2):
        ax.plot(grid_param_1, scores_mean[idx,:], '-o', label= name_param_2 + ': ' + str(val))

    ax.set_title("Grid Search Scores", fontsize=20, fontweight='bold')
    ax.set_xlabel(name_param_1, fontsize=16)
    ax.set_ylabel('CV Average Score', fontsize=16)
    ax.legend(loc="best", fontsize=15)
    ax.grid('on')

# Calling Method 
plot_grid_search(pipe_grid.cv_results_, n_estimators, max_features, 'N Estimators', 'Max Features')